In [1]:
import pandas as pd
pd.set_option('display.max_rows',500)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
plt.style.use("fivethirtyeight")
%matplotlib inline

# For reading stock data from yahoo
import pandas_datareader.data as pdr
from pandas_datareader.data import DataReader
import yfinance as yf

# For time stamps
import datetime

from prophet import Prophet

C:\Users\0074fasaet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [36]:
# download stocks
file = pd.read_csv('M6_Universe.csv')
stocks_list = file['symbol'].values.tolist()
stocks_list.remove('DRE')

end = datetime.datetime.now()
start = datetime.datetime(end.year - 3, end.month, end.day)

last_month_days = []
for year in range(start.year, end.year+1):
    for month in range(1,13):
        first_day = datetime.datetime(year, month, 1)
        prev_day = first_day - datetime.timedelta(days=1)
        last_month_days += [prev_day.strftime('%Y-%m-%d')]

In [35]:
(start, end)

(datetime.datetime(2019, 10, 14, 0, 0),
 datetime.datetime(2022, 10, 14, 13, 59, 25, 783331))

## Download 100 assets

In [28]:
all_stocks = DataReader(stocks_list, 'yahoo', start, end)['Adj Close']

all_stocks.describe()

C:\Users\0074fasaet\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas_datareader\base.py:272: SymbolWarning: Failed to read symbol: 'DRE', replacing with NaN.
  warnings.warn(msg.format(sym), SymbolWarning)


Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX,DRE
count,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,...,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,758.000000,0.0
mean,105.772335,262.890544,84.601859,140.175229,113.507094,96.879613,215.341318,141.837905,189.054207,161.383127,...,114.601855,50.441648,154.564186,87.047300,63.674691,62.486270,65.302815,70.602788,60.525917,NaN
std,26.627506,59.653083,8.181538,25.295222,14.790311,34.865303,63.212270,28.659747,33.847146,38.754613,...,15.635187,16.419204,27.374767,13.678612,11.481049,6.407882,7.046190,12.941520,45.975439,NaN
min,56.890747,138.345779,64.298126,76.798355,76.101143,37.084087,78.741394,83.830498,110.256180,75.713036,...,71.814056,20.812737,85.449226,46.945171,39.308060,41.590977,45.549580,36.500801,17.629999,NaN
25%,82.476305,211.458389,78.967226,121.218027,100.258615,60.202109,154.236931,117.634129,156.329678,124.350616,...,99.939125,35.977672,131.873497,77.251863,54.294428,57.771837,59.370608,57.831249,24.582500,NaN
50%,102.121258,263.981110,83.136578,145.525475,112.125065,98.728855,231.982750,154.406998,192.040199,166.233093,...,118.257614,48.704531,156.440292,88.181301,61.019869,62.390457,66.482521,73.703140,41.280001,NaN
75%,130.488258,311.054504,88.104332,159.075439,126.969057,131.315964,268.626953,164.609997,216.045387,198.659988,...,129.165131,61.035776,176.383125,99.481842,75.155302,66.732855,70.947666,81.894413,80.820000,NaN
max,170.146515,409.828949,105.180000,191.601440,145.579803,165.868637,324.398621,186.570496,252.427429,223.893570,...,141.664886,90.272392,209.854370,105.722588,85.122887,77.594452,79.452660,89.421799,276.000000,NaN


In [37]:
all_stocks

Symbols,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX,DRE
Date,,,,,,,,,,,,,,,,,,,,,
2019-10-14,64.191063,177.151794,83.316025,118.960152,96.765266,50.367996,130.328644,86.821503,197.139618,107.534836,...,85.191994,48.288567,117.385376,72.851387,48.418880,57.756374,55.924145,53.539558,89.080002,NaN
2019-10-15,64.835052,178.584305,83.053474,118.723564,98.056122,51.308632,131.643005,88.369003,196.497803,108.220619,...,86.746117,48.488865,118.417725,73.440674,49.247795,57.574497,55.683971,53.915672,86.760002,NaN
2019-10-16,65.052620,178.939178,83.832054,118.714104,99.616699,49.999493,131.220535,88.871498,198.161102,108.658752,...,86.632385,47.812859,118.904694,73.402641,49.335564,57.701820,55.739403,54.028507,85.680000,NaN
2019-10-17,65.392036,178.430832,84.194191,119.177803,100.310295,50.561947,131.896500,89.374001,198.043610,109.868416,...,87.276787,47.846245,119.216331,73.763817,49.628124,57.847317,55.998051,54.169544,85.160004,NaN
2019-10-18,66.540787,179.428284,85.226234,118.922287,100.136894,50.115864,133.314133,87.875504,199.028931,111.335236,...,86.954605,47.595867,118.982590,73.127014,49.052761,58.038296,56.108902,54.075520,84.959999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-10,136.916809,256.708435,84.449997,149.570007,91.790001,79.190002,272.899994,113.669998,170.860001,165.380005,...,121.849998,80.120003,139.779999,85.459999,48.599998,63.869999,66.739998,69.610001,21.430000,NaN
2022-10-11,140.084579,251.859985,84.870003,150.110001,91.580002,76.300003,266.290009,112.209999,174.710007,166.369995,...,122.500000,79.480003,138.639999,85.440002,47.689999,63.639999,67.290001,69.070000,21.760000,NaN
2022-10-12,138.569992,250.070007,81.739998,146.559998,90.500000,76.010002,263.399994,112.900002,171.669998,163.460007,...,121.900002,80.120003,139.000000,84.769997,47.630001,61.520000,67.650002,68.550003,21.540001,NaN


In [38]:
# extract monthly returns
month_stocks = all_stocks.copy()
month_stocks['Datestring'] = month_stocks.index
    
month_stocks['year'] = pd.DatetimeIndex(month_stocks.index).year
month_stocks['month'] = pd.DatetimeIndex(month_stocks.index).month

month_on_month = month_stocks.sort_values(by='Date').groupby(['year','month']).last()
month_on_month = month_on_month.drop(month_on_month.tail(1).index).set_index(['Datestring'])

monthly_returns = month_on_month.pct_change().round(3).T
monthly_returns

Datestring,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,2020-04-30,2020-05-29,2020-06-30,2020-07-31,...,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-29,2022-05-31,2022-06-30,2022-07-29,2022-08-31,2022-09-30
Symbols,,,,,,,,,,,,,,,,,,,,,
ABBV,NaN,0.103,0.009,-0.073,0.058,-0.111,0.095,0.127,0.059,-0.022,...,0.175,0.021,0.079,0.097,-0.086,0.003,0.039,-0.054,-0.063,-0.002
ACN,NaN,0.085,0.047,-0.022,-0.120,-0.096,0.139,0.089,0.065,0.051,...,0.160,-0.145,-0.106,0.067,-0.107,-0.006,-0.070,0.107,-0.058,-0.108
AEP,NaN,-0.025,0.035,0.103,-0.138,-0.104,0.039,0.035,-0.066,0.091,...,0.098,0.016,0.012,0.101,-0.007,0.038,-0.060,0.027,0.025,-0.137
AIZ,NaN,0.059,-0.013,-0.004,-0.072,-0.137,0.021,-0.028,0.007,0.040,...,0.025,-0.021,0.117,0.071,0.000,-0.025,-0.022,0.017,-0.095,-0.083
ALLE,NaN,0.034,0.040,0.038,-0.111,-0.197,0.093,-0.008,0.028,-0.027,...,0.074,-0.073,-0.067,-0.038,0.041,-0.023,-0.120,0.080,-0.100,-0.053
AMAT,NaN,0.071,0.054,-0.050,0.005,-0.212,0.084,0.135,0.076,0.064,...,0.069,-0.122,-0.027,-0.018,-0.163,0.065,-0.224,0.165,-0.110,-0.129
AMP,NaN,0.093,0.017,-0.007,-0.141,-0.275,0.122,0.229,0.071,0.024,...,0.042,0.009,-0.011,0.002,-0.116,0.045,-0.140,0.136,-0.002,-0.060
AMZN,NaN,0.014,0.026,0.087,-0.062,0.035,0.269,-0.013,0.130,0.147,...,-0.049,-0.103,0.027,0.061,-0.238,-0.033,-0.116,0.271,-0.061,-0.109
AVB,NaN,-0.015,-0.015,0.033,-0.074,-0.259,0.107,-0.043,0.002,-0.010,...,0.064,-0.033,-0.023,0.047,-0.084,-0.086,-0.058,0.101,-0.061,-0.075


In [39]:
# Generate monthly return forecast table
future_steps = 4
first_time = True
for ticker in stocks_list:
    print('Ticker: %s...' % ticker)
    df = monthly_returns.T[ticker].dropna().reset_index().rename(columns={'Datestring':'ds', ticker:'y'})
    model = Prophet(interval_width=0.95).fit(df)
    future_dates = model.make_future_dataframe(periods=future_steps, freq='M')
    forecast = model.predict(future_dates)
    if first_time:
        forecast_df = forecast[['ds']].set_index('ds')
        lower_df = forecast_df.copy()
        yhat_df = forecast_df.copy()
        upper_df = forecast_df.copy()
        first_time = False

    forecast = forecast.set_index('ds')

    lower_df[ticker] = forecast['yhat_lower']
    yhat_df[ticker] = forecast['yhat']
    upper_df[ticker] = forecast['yhat_upper']
        

14:01:13 - cmdstanpy - INFO - Chain [1] start processing


Ticker: ABBV...


14:01:14 - cmdstanpy - INFO - Chain [1] done processing
14:01:14 - cmdstanpy - INFO - Chain [1] start processing


Ticker: ACN...


14:01:14 - cmdstanpy - INFO - Chain [1] done processing
14:01:15 - cmdstanpy - INFO - Chain [1] start processing


...

14:02:14 - cmdstanpy - INFO - Chain [1] done processing
14:02:15 - cmdstanpy - INFO - Chain [1] start processing


Ticker: XLB...


14:02:15 - cmdstanpy - INFO - Chain [1] done processing
14:02:15 - cmdstanpy - INFO - Chain [1] start processing


Ticker: VXX...


14:02:15 - cmdstanpy - INFO - Chain [1] done processing


In [40]:
lower_df

,ABBV,ACN,AEP,AIZ,ALLE,AMAT,AMP,AMZN,AVB,AVY,...,XLF,XLV,XLE,XLY,XLI,XLC,XLU,XLP,XLB,VXX
ds,,,,,,,,,,,,,,,,,,,,,
2019-11-29,-0.003297,-0.017909,-0.096292,-0.029853,-0.073331,-0.090544,-0.046873,-0.081453,-0.124691,-0.068392,...,-0.071137,-0.010023,-0.195037,-0.075640,-0.059069,-0.034996,-0.081040,-0.040887,-0.058733,-0.463947
2019-12-31,-0.068791,-0.048364,-0.024611,-0.099723,-0.068761,-0.124834,-0.159151,-0.089516,-0.130586,-0.074043,...,-0.102559,-0.023735,-0.207673,-0.085144,-0.101809,-0.052805,-0.020895,-0.006722,-0.064577,-0.334874
2020-01-31,-0.108176,-0.129267,-0.042246,-0.089833,-0.114391,-0.114723,-0.075686,-0.046617,-0.091858,-0.099822,...,-0.091292,-0.072543,-0.127444,-0.073091,-0.095298,-0.041665,-0.050486,-0.057952,-0.111138,-0.208941
2020-02-28,-0.018407,-0.131991,-0.141250,-0.072395,-0.142494,-0.038914,-0.113854,-0.047957,-0.108544,-0.147222,...,-0.110204,-0.068888,-0.179750,-0.079556,-0.097005,-0.057368,-0.118780,-0.088353,-0.072999,-0.187190
2020-03-31,-0.158420,-0.080406,-0.087416,-0.108273,-0.144564,-0.163256,-0.255792,-0.102375,-0.190335,-0.087916,...,-0.182943,-0.043143,-0.369438,-0.121723,-0.149167,-0.108621,-0.065227,-0.050277,-0.111820,0.085062
2020-04-30,0.004239,0.058803,-0.023068,-0.019779,-0.001812,-0.042122,0.037041,0.156920,0.009358,0.062491,...,0.025755,0.059431,0.028146,0.105237,-0.002718,0.083441,-0.008192,0.004328,0.057848,-0.570928
2020-05-29,-0.034545,-0.030245,-0.074759,-0.086766,-0.072431,0.018741,0.010650,-0.073637,-0.107335,-0.067776,...,-0.041422,-0.020740,-0.121888,-0.038771,-0.032209,0.005272,-0.050188,-0.044056,-0.008465,-0.445818
2020-06-30,-0.051822,-0.011769,-0.116252,-0.102606,-0.073715,-0.023219,-0.052088,0.031503,-0.085023,-0.063274,...,-0.101742,-0.063235,-0.199531,-0.015321,-0.069195,-0.035522,-0.087961,-0.048320,-0.072198,-0.397994
2020-07-31,-0.082245,-0.040392,0.009900,-0.068190,-0.156505,-0.102963,-0.096138,-0.023739,-0.092040,-0.118493,...,-0.074723,0.006566,-0.260969,-0.035072,-0.064864,0.002709,0.007586,0.003985,-0.022185,-0.466605


In [41]:
# Quantile table generation
timestamp = '2022-10-31'
quantile_table = pd.DataFrame([],columns=stocks_list).T
quantile_table['5'] = np.nan
quantile_table['y_hat'] = np.nan
quantile_table['95'] = np.nan

quantile_table = quantile_table.T

for ticker in stocks_list:
    q_5 = lower_df[ticker][timestamp]
    q_y = yhat_df[ticker][timestamp]
    q_95 = upper_df[ticker][timestamp]

    quantile_table[ticker] = [q_5, q_y, q_95]
    
    
quantile_table = quantile_table.T

# 95th percentile is 3 std away from mean
quantile_table['f_std'] = (quantile_table['95'] - quantile_table['y_hat']) / 3

quantile_table

,5,y_hat,95,f_std
ABBV,-0.136964,-0.050546,0.029699,0.026749
ACN,-0.249459,-0.164972,-0.081110,0.027954
AEP,0.041945,0.119009,0.201713,0.027568
AIZ,-0.125306,-0.035079,0.060172,0.031750
ALLE,-0.161187,-0.060499,0.045476,0.035325
AMAT,-0.258980,-0.134390,-0.017967,0.038808
AMP,-0.145680,-0.039927,0.067520,0.035815
AMZN,-0.259308,-0.177162,-0.086984,0.030059
AVB,-0.228887,-0.130023,-0.035857,0.031389
AVY,-0.061868,0.019264,0.106119,0.028952


In [42]:
# generate table of sampling
sampling_table = quantile_table.copy().drop(quantile_table.columns, axis=1)
for i in range(1,1001):
    sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
    
sampling_table = sampling_table.round(4)
sampling_table

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

...

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\3857847923.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  sampling_table['s_%d' % i] = np.random.normal(loc=quantile_table['y_hat'], scale=quantile_table['f_std'], size=None)
C:\Users\0074fasaet\AppData\Local\Temp\ipykernel_17792\385784792

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_991,s_992,s_993,s_994,s_995,s_996,s_997,s_998,s_999,s_1000
ABBV,-0.0520,-0.0680,-0.0717,-0.0572,-0.0126,-0.0559,-0.0683,-0.0960,-0.1245,-0.0794,...,-0.0526,-0.0386,-0.0573,-0.0367,-0.0766,-0.0674,-0.0769,-0.0370,-0.0057,-0.0358
ACN,-0.1595,-0.1484,-0.1045,-0.1528,-0.1953,-0.1238,-0.2051,-0.0758,-0.1330,-0.1451,...,-0.1256,-0.1923,-0.2147,-0.1737,-0.1710,-0.1524,-0.1927,-0.1642,-0.1671,-0.1859
AEP,0.1045,0.1041,0.1394,0.1196,0.1426,0.1582,0.1310,0.1560,0.1415,0.1341,...,0.0633,0.1200,0.1096,0.0694,0.1380,0.1183,0.1320,0.1215,0.1226,0.1142
AIZ,-0.0617,-0.0192,-0.0566,-0.0963,-0.0339,-0.0903,0.0107,-0.0582,-0.0280,-0.0228,...,-0.0289,-0.0201,0.0038,-0.0134,0.0010,0.0130,-0.0988,-0.0225,-0.0418,0.0074
ALLE,-0.1493,-0.0398,-0.0442,-0.0780,-0.0376,-0.0868,-0.0557,-0.1188,-0.0574,-0.0233,...,-0.0998,-0.0329,-0.0973,-0.0651,-0.0815,-0.0317,-0.1307,-0.0544,-0.0475,-0.1060
AMAT,-0.1980,-0.1519,-0.1508,-0.1721,-0.1822,-0.1158,-0.1012,-0.1647,-0.1179,-0.1139,...,-0.0867,-0.0827,-0.1453,-0.1220,-0.1480,-0.1802,-0.2005,-0.1342,-0.0617,-0.1801
AMP,-0.0038,-0.0278,-0.0385,-0.0516,-0.0343,-0.0131,-0.0160,0.0516,-0.0290,-0.0859,...,-0.0556,-0.1096,-0.0272,-0.0520,-0.0358,-0.0726,-0.0616,-0.0118,-0.0140,-0.0279
AMZN,-0.1827,-0.1648,-0.1638,-0.1543,-0.1858,-0.1654,-0.1626,-0.1701,-0.2037,-0.1755,...,-0.1797,-0.2196,-0.1270,-0.1385,-0.1363,-0.1985,-0.1657,-0.1703,-0.1633,-0.1598
AVB,-0.1472,-0.0977,-0.0241,-0.1344,-0.1854,-0.0830,-0.1639,-0.1247,-0.1322,-0.1470,...,-0.0853,-0.1192,-0.1360,-0.1241,-0.1500,-0.1487,-0.1296,-0.0940,-0.0749,-0.1617
AVY,0.0027,0.0913,0.0089,0.0020,0.0693,0.0281,-0.0177,0.0042,0.0491,0.0246,...,0.0409,0.0113,-0.0102,0.0185,-0.0414,0.0511,-0.0153,0.0506,0.0539,0.0377


In [43]:
# extract total ranks
def generate_rank(df):
    ranks = pd.DataFrame(index=df.index, columns=df.columns)
    vector_ranks = pd.DataFrame(index=df.index, columns=df.columns)
    
    for col in df.columns:
        if df[col].isnull().all():
            continue
        ranks[col], vector_ranks[col] = generate_rank_from_values(df[col].values)
        
    return ranks, vector_ranks

def generate_rank_from_values(value_list):
    if len(value_list) != 99:
        print('Values list not 99 in number')
        return
    df = pd.DataFrame(value_list, columns=['data'])
    orig_index = df.index
    #df[241] = [random.randint(1,20) for i in range(100)]
    df = df.sort_values('data',ascending=False)
    df['my100rank'] = range(99,0,-1)
    df['my5rank'] = [5]*20 + [4]*20 + [3]*20 + [2]*20 + [1]*19
    df['pyrank'] = df['data'].rank()
    df['5rank'] = pd.qcut(df['data'], 5, labels=False, precision=1) + 1
    df['diff'] = df['my5rank'] != df['5rank']
    uniq_ranks = df[df['diff'] == True]['pyrank'].unique()
    df['adjusted5rank'] = df['my5rank']
    df['rankvector'] = np.nan
    
    df = df.join(pd.get_dummies(df['my5rank']))
    
    
    # ties on the margins of the classes
    for rank in uniq_ranks:
        rank_index = df[df['pyrank']==rank].index
        population = df['my5rank'][rank_index]
        df.loc[rank_index,'adjusted5rank'] = round(population.mean(),2)
        for p in population:
            df.loc[rank_index, p] = round(sum(population == p) / len(population),2)
            
    
    
    #print(df.to_string())
    ranks = df['adjusted5rank'][orig_index].values.tolist()
    vector_ranks = []
    for index, row in df.loc[orig_index].iterrows():
        vector_ranks.append([row[1],row[2],row[3],row[4],row[5]])
    
    # clean house
    for p in [1,2,3,4,5]:
        df[f'Rank {p}'] = df[p]
    df = df.drop(columns=[1,2,3,4,5])
    return ranks, vector_ranks

ranks, vector_ranks = generate_rank(sampling_table)
ranks

,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,s_10,...,s_991,s_992,s_993,s_994,s_995,s_996,s_997,s_998,s_999,s_1000
ABBV,4,3,3,4,4,4,3,2,2,3,...,4,4,3,4,3,3,3,4,5,4
ACN,1,2,2,2,1,2,1,3,2,1,...,2,1,1,1,1,1,1,1,1,1
AEP,5,5,5,5,5,5,5,5,5,5,...,5,5,5,5,5,5,5,5,5,5
AIZ,3,5,4,2,4,3,5,4,4,4,...,4,4,5,4,5,5,3,4,4,5
ALLE,2,4,4,3,4,3,4,2,3,4,...,3,4,3,3,3,4,2,4,4,2
AMAT,1,2,2,1,1,2,2,1,2,2,...,3,3,2,2,2,1,1,2,3,1
AMP,5,4,4,4,4,5,4,5,4,3,...,4,2,4,4,4,3,4,5,5,4
AMZN,1,1,1,1,1,1,1,1,1,1,...,1,1,2,2,2,1,1,1,1,2
AVB,2,2,4,2,1,3,1,2,2,1,...,3,2,2,2,2,2,2,3,3,1
AVY,5,5,5,4,5,5,4,5,5,5,...,5,5,5,5,4,5,5,5,5,5


In [45]:
pmf_vranks = []

for row in ranks.index:
    pmf, bins = np.histogram(ranks.T[row].dropna().values - 0.001, bins = range(0,6), density=True)
    pmf = np.round(pmf, 3).tolist()
    pmf_vranks.append(pmf)
    

ranks_table = ranks.copy().drop(ranks.columns, axis=1)

# rank histogram per instrument
ranks_table['PMF Rank Vector'] = pmf_vranks
# expected rank
ranks_table['PMF Rank'] = np.dot(np.array(pmf_vranks), np.array(range(1,6)))
ranks_table

,PMF Rank Vector,PMF Rank
ABBV,"[0.0, 0.039, 0.369, 0.504, 0.088]",3.641
ACN,"[0.704, 0.289, 0.007, 0.0, 0.0]",1.303
AEP,"[0.0, 0.0, 0.0, 0.0, 1.0]",5.000
AIZ,"[0.0, 0.03, 0.261, 0.48, 0.229]",3.908
ALLE,"[0.009, 0.166, 0.393, 0.34, 0.092]",3.340
AMAT,"[0.302, 0.532, 0.138, 0.027, 0.001]",1.893
AMP,"[0.001, 0.067, 0.269, 0.409, 0.254]",3.848
AMZN,"[0.828, 0.171, 0.001, 0.0, 0.0]",1.173
AVB,"[0.231, 0.626, 0.13, 0.013, 0.0]",1.925
AVY,"[0.0, 0.0, 0.005, 0.129, 0.866]",4.861


In [46]:
# break up single column vector array to separate columns
pmf_values = np.array([x for x in ranks_table['PMF Rank Vector'].values])
submission = ranks_table.filter('PMF Rank Vector')
for i in range(5):
    submission[f'Rank {i+1}'] = pmf_values[:,i]

    
diff = 1 - submission[['Rank 1','Rank 2','Rank 3','Rank 4','Rank 5']].sum(1)
submission['Rank 5'] = submission['Rank 5'] + diff

# portfolio strategy
submission['Decision'] = submission['Rank 5'] / submission['Rank 5'].sum()
# round up to 5 dp
submission = submission.round(5)

diff = 1 - submission['Decision'].sum()
submission['Decision'][-1] = submission['Decision'][-1] + diff



submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.000,0.039,0.369,0.504,0.088,0.00440
ACN,0.704,0.289,0.007,0.000,0.000,0.00000
AEP,0.000,0.000,0.000,0.000,1.000,0.04997
AIZ,0.000,0.030,0.261,0.480,0.229,0.01144
ALLE,0.009,0.166,0.393,0.340,0.092,0.00460
AMAT,0.302,0.532,0.138,0.027,0.001,0.00005
AMP,0.001,0.067,0.269,0.409,0.254,0.01269
AMZN,0.828,0.171,0.001,0.000,0.000,0.00000
AVB,0.231,0.626,0.130,0.013,0.000,0.00000
AVY,0.000,0.000,0.005,0.129,0.866,0.04328


In [47]:
# Save to file
submission.to_csv('submission.csv')
submission

,Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Decision
ABBV,0.000,0.039,0.369,0.504,0.088,0.00440
ACN,0.704,0.289,0.007,0.000,0.000,0.00000
AEP,0.000,0.000,0.000,0.000,1.000,0.04997
AIZ,0.000,0.030,0.261,0.480,0.229,0.01144
ALLE,0.009,0.166,0.393,0.340,0.092,0.00460
AMAT,0.302,0.532,0.138,0.027,0.001,0.00005
AMP,0.001,0.067,0.269,0.409,0.254,0.01269
AMZN,0.828,0.171,0.001,0.000,0.000,0.00000
AVB,0.231,0.626,0.130,0.013,0.000,0.00000
AVY,0.000,0.000,0.005,0.129,0.866,0.04328
